<div style="text-align: center;">

<h1 style= "color:yellowgreen">PID Control
</h1>

</div>


<h2 style= "color:coral">Intro to PID
</h2>

In notebook 08c, we saw that only having a proportional gain is not sufficient to handle different uncertainties in the system.  
We need more parameters to adjust our control input.  
First, let us review some concepts in control theory.

When we are looking at the response of the controller, we will see a response like the image below:
- $x_d$ - desired value
- $x$ - current sensor position
- $\dot{x}$ -  velocity

<br><img src="../assets/ex8-imgs/pid-response.png" style="width: 40%; height: auto;"></br>

There are some control theory terms related to this system response:
- Overshoot: percentage past the desired value
- Settling time: time that it takes for $x$ to reach "close" to $x_d$, e.g., 90%
- Rise time: Time that takes for position to go from 10% to 90% of $x_d$
- Steady state error: error after the initial reaction (after settling time)


Now, regading the new parameter need for our controller.
Intuitively, we need:
- Something that responds to error
	- *Proportional gain to the error* $k_p$
- Something that reduces steady state error
	- Need to consider the accumulated error of the system
	- Accumulate implies integral
	- *Integral gain* $k_i$
- Something that can speed up the controller response and control the overshoot
	- Need to consider the slope of the error. 
	- Slope implies derivative
	- *Derivative gain* $k_d$

In this manner, we have three gains to tune.  
Formally, the control input function of a PID controller with error $e$ is defined as:

$$control\_input (t) = k_p e(t) + k_i \int e(t) + k_d \frac{d}{dt} e(t)$$

In the feedback control block diagram, we have:

<br><img src="../assets/ex8-imgs/pid-block-diagram.png" style="width: 40%; height: auto;"></br>

<span  style= "color:darkturquoise"> 

We have to tune gains $k_p$, $k_i$, and $k_d$

</span>

But how??



<h2 style= "color:coral">Example Problem</h2>

You want to program an autonomous car to stop EXACTLY at the stop sign

<br><img src="../assets/ex8-imgs/pid-example.png" style="width: 40%; height: auto;"></br>

We control the car via the accelerator - speeding up: positive acceleration and braking: negative acceleration.

Acceleration dynamics model of the car:

$$\ddot{x} = \frac{F_{control}}{m} - sign(\dot{x})(\mu g +\frac{c\rho A \dot{x}}{2m})$$

- Integrate once: we have the car's velocity
- Integrate twice: we have the car's position

That's our dynamic system.  
The information of the system is given by: $(x,\dot{x})$ - (position, velocity) and control input
Let's tune the a PID controller with the goal of reaching position $30$ meters (stop sign)

<h3 style= "color:coral"> Tuning proportional gain</h3>


- System starting in position $(0,0)$ and control input $0$
<br><img src="../assets/ex8-imgs/pid-no-input.png" style="width: 40%; height: auto;"></br>
Not much since we are not accelerating or braking the car.

- System with initial velocity $(0,20 m/s)$
<br><img src="../assets/ex8-imgs/pid-initial-vel.png" style="width: 40%; height: auto;"></br>
The vehicle will move until it naturally deaccelarate reaching position $100$ meters

- Alright, let add a proportional controller with $k_p = 1$ with vehicle starting $(0,0)$
<br><img src="../assets/ex8-imgs//pid-kp1.png" style="width: 40%; height: auto;"></br>
Okay. We get closer to our desired position. 
However, there are still quite a bit of oscillations and steady-state error

- How about $k_p = 2$
<br><img src="../assets/ex8-imgs/pid-kp2.png" style="width: 40%; height: auto;"></br>
Ohh, we made it worse.

- Reduce $k_p$, $k_p =0.5$ 
<br><img src="../assets/ex8-imgs/pid-kp05.png" style="width: 40%; height: auto;"></br>
Better! So reducing $k_p$ produces better results. 

- Let's try $k_p = 0.15$ 
<br><img src="../assets/ex8-imgs/pid-kp015.png" style="width: 40%; height: auto;"></br>
Good!

- How about reducing more: $k_p =0.1$ 
<br><img src="../assets/ex8-imgs/pid-kp01.png" style="width: 40%; height: auto;"></br>
Oh, not strong enough. Steady state error got worse.  
Okay let's keep $k_p = 0.15$ then.  
Next, let's get remove the steady state error

<h3 style= "color:coral"> Tuning integral gain</h3>

We already found $k_p = 0.15$ as our proportional gain.

- Start with $k_i = 0.1$
<br><img src="../assets/ex8-imgs/pid-ki01.png" style="width: 40%; height: auto;"></br>
Oh no! We made it unstable.

- Smaller value for $k_i$: $k_i = 0.01$
<br><img src="../assets/ex8-imgs/pid-ki001.png" style="width: 40%; height: auto;"></br>
Much better!   
Steady state error has been removed.  
However, we have a large overshoot and our response is slow.   
Next, tune $k_d$ to adjust the response.  

<h3 style= "color:coral"> Tuning derivative gain</h3>

- Start with $k_d = 0.1$ 
<br><img src="../assets/ex8-imgs/pid-kd01.png" style="width: 40%; height: auto;"></br>
Reduced overshot but still slow

- Increase $k_d$: $k_d = 0.4$
<br><img src="../assets/ex8-imgs/pid-kd04.png" style="width: 40%; height: auto;"></br>
Looks great!!
Overshoot has been eliminated, rise-time is fast, and no steady state error!

There we go, we found our PID: $k_p = 0.15, k_i = 0.01, k_d = 0.4$


<h3 style= "color:coral"> Manual Tuning</h3>

- Increase $k_p$ unstil system oscillates once (one peak is fine)
- Increase $k_i$ until system converges to desired value (if needed)
- Increase $k_d$ to reduce overshoot and settling time


<h3 style= "color:coral"> Model-based</h3>

In EE380 and EE482, you will learn methods to design PID controllers using the system model.  
The design method will provide the values to satisfy desired responses.



<h6><p style="text-align: center;">
<strong>Disclaimer:</strong>
Tutorial is an adaptation of the Prof. Robinette's EECE5560 UMass Lowell class.
Images from: <strong> <a href="https://studentuml-my.sharepoint.com/:p:/g/personal/paul_robinette_uml_edu/EakZv6Pq415Ls4dcKQpvOMUBPiQ0moEYR14gFKw7EXTRiw?rtime=YNt168793Eg">Slides</a></strong>
</p></h6>